# Insertion algorithm 

In [188]:
%matplotlib inline

import numpy as np
import os 
import math 
import matplotlib.pyplot as plt
import matplotlib
import sys 
import torch 
import signatory
import itertools

## Insertion operator 

In [113]:
test_path = torch.rand((1,10,5))

print(test_path)

tensor([[[0.8374, 0.2912, 0.7036, 0.4183, 0.5546],
         [0.5598, 0.4505, 0.3918, 0.0244, 0.9650],
         [0.0223, 0.4697, 0.8233, 0.8471, 0.3236],
         [0.4044, 0.2251, 0.7779, 0.3750, 0.4144],
         [0.5029, 0.3032, 0.5652, 0.1450, 0.8820],
         [0.6511, 0.8199, 0.0190, 0.9331, 0.1870],
         [0.4765, 0.6081, 0.5612, 0.2418, 0.2141],
         [0.8928, 0.9133, 0.1162, 0.8548, 0.4311],
         [0.7902, 0.3508, 0.6837, 0.4870, 0.0934],
         [0.8029, 0.5617, 0.2014, 0.9772, 0.4055]]])


In [309]:
signature_test = signatory.signature(test_path, 2)
print(signature_test.size())

torch.Size([1, 30])


In [128]:
def get_signature_as_tensor(sig,dimension,order):
    
    ''' This function transforms a signature given as a vector into a series of tensors, 
    stored in a dictionnary. It uses the method extract_signature_term() of Signatory. 
    
    For instance, for a path in R^d, the key called 'Depth z' is a Pytorch tensor 
    of size (R^d)^{\otimes z}.
    
    Arguments that need to be specified are:
        - sig : the original signature
        - dimension : the dimension of the original path
        - order : the order (or total depth) at which the original signature is calculated.
    
    '''
    
    total_tensor = {}
    
    for depth in np.arange(1,order+1):
        
        signature_at_depth = signatory.extract_signature_term(sig,dimension,int(depth))
        
        tensor_size = [dimension]*int(depth)
        
        signature_at_depth = signature_at_depth.view(tensor_size)
        
        total_tensor["Depth " + str(depth)] = signature_at_depth
        
    return total_tensor      

In [470]:
get_signature_as_tensor(signature_test,5,2)

{'Depth 1': tensor([-0.0345,  0.2705, -0.5022,  0.5589, -0.1491]),
 'Depth 2': tensor([[ 0.0006, -0.0441,  0.0014, -0.1432,  0.1836],
         [ 0.0347,  0.0366,  0.0377,  0.2514, -0.2213],
         [ 0.0159, -0.1736,  0.1261, -0.3345,  0.2081],
         [ 0.1239, -0.1002,  0.0537,  0.1562, -0.1871],
         [-0.1785,  0.1810, -0.1332,  0.1037,  0.0111]])}

In [582]:
def Insertion(p,n,x,signature,dimension):
    
    '''This function computes the Insertion operator taken at x, for parameters p and n.
    
    Arguments : 
    
        - x : vector in R^dimension at which the operator should be evaluated.
        - p : insertion spot. 
        - n : total depth of the signature.
        - signature : total signature of a path
        - dimension : dimension of the path.
    '''
    
    #Get the last (n-th) term of the signature as a tensor of size (R^dimension)^{\otimes n}
    
    last_signature_term = get_signature_as_tensor(signature,dimension,n)["Depth "+ str(n)]
    
    last_signature_term = torch.ones_like(last_signature_term)
    
    #Add a dimension, since the output of the operator should be in (R^d)^{\otimes (n+1)}
    
    new_tensor = last_signature_term.unsqueeze(0).clone()
    new_tensor = new_tensor.expand([dimension]*(n+1))
    
    #Creates a list containing all possible coordinates of the big tensor.
    #This list is of length dimension**n. 
    
    tensor_size = np.arange(dimension)
    
    coordinates_list = list(itertools.product(tensor_size,repeat= n+1))
    
    
    #Every element of coordinates_list is a coordinate of the new tensor, which has d^(n+1) coordinates.
    
    #Computes the Insertion operator.
    
    print(new_tensor[1,1,1])
    
    for coordinate in coordinates_list:
        
        coordinate = list(coordinate)
        
        p_inverse = n-p+1
        
        x_coordinate = x[coordinate[p_inverse]]
        
        new_coordinate = coordinate.copy()
        
        del new_coordinate[p_inverse]
        
        new_tensor[tuple(coordinate)] = last_signature_term[tuple(new_coordinate)]*x_coordinate
        
        
    return new_tensor

In [588]:
x=[1,2,3,4,5]

p=3

n=2

dimension=5

Insertion(p,n,x,signature_test,dimension)

tensor(1.)


tensor([[[5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.]],

        [[5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.]],

        [[5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.]],

        [[5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.]],

        [[5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.],
         [5., 5., 5., 5., 5.]]])

In [361]:
def get_length(signature,dimension,order):
    
    '''This function approximates the length of the path through the signature.
    
        Arguments are : 
            - signature : the full signature. 
            - order : the order at which the signature was truncated. 
    
    '''
    
    last_signature_term = signatory.extract_signature_term(signature,dimension,int(order))
    
    return torch.norm(math.factorial(order)*last_signature_term,2)*(1/order)

In [363]:
get_length(signature_test,5,2)

tensor(0.7448)

In [431]:
def get_A_matrix(p,signature,order,dimension):
    
    '''This function creates the matrix lenght_of_path*A, used in the optimization problem.
    
        Arguments: 
            - p : insertion spot.
            - signature : full original signature.
            - order : order of truncation of signature. 
            - dimension : dimension of the path. 
    
    '''
    
    #Create basis of R^d
    
    basis = np.eye(dimension,dimension)
    
    #Evaluate the insertion operator on the basis
    
    total_tensor = torch.zeros(dimension,dimension**2,dimension)
    
    for row in np.arange(dimension):
    
        base_vector = basis[row,:]
        
        insertion_tensor = Insertion(p,order,base_vector,signature,dimension)
        
        print(insertion_tensor.size())
        
        insertion_tensor = insertion_tensor.reshape(-1,dimension)
        
        total_tensor[row,:,:] = insertion_tensor
        
    
    total_tensor = total_tensor.reshape(dimension**(order+1),dimension)
    
    tensor_size = str(list(total_tensor.size()))
    
    length = get_length(signature,dimension,order)
    
    print('Size of A matrix is ' + tensor_size + ', should be [' + str(dimension**(order+1)) +','+ str(dimension)+']')
    
    return length*total_tensor

    


In [430]:
get_A_matrix(1,signature_test,2,5)

torch.Size([5, 5, 5])
torch.Size([5, 5, 5])
torch.Size([5, 5, 5])
torch.Size([5, 5, 5])
torch.Size([5, 5, 5])
Size of A matrix is [125, 5], should be [125,5]


tensor([[0.7448, 0.7448, 0.7448, 0.7448, 0.7448],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7448, 0.7448, 0.7448, 0.7448, 0.7448],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7448, 0.7448, 0.7448, 0.7448, 0.7448],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7448, 0.7448, 0.7448, 0.7448, 0.7448],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
